In [1]:
import jieba
import pandas as pd
import codecs
import string
import re
from sklearn.metrics import silhouette_score,silhouette_samples
import matplotlib.cm as cm
import numpy as np
import jieba.posseg as jp
from gensim import corpora, models

In [2]:
# 清洗文本
def clearTxt(line:str):
    if(line != ''):
        line = line.strip()
        # 去除文本中的中文符号和英文符号
        rule = re.sub("[a-zA-Z0-9]","",line)
        line = re.sub("[\s+\.\!\/_,$%^*(+\"\'；：“”．]+|[+——！，。？?、~@#￥%……&*（）]+", "",rule)
        return line
    return None

#文本切割
def sent2word(line):
    segList = jieba.cut(line,cut_all=False)
    segSentence = ''
    for word in segList:
        if word != '\t':
            segSentence += word + " "
    return segSentence.strip()

def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding="GB18030").readlines()]  
    return stopwords  


def remove_stopwords(ls):  # 去除停用词
    return [word for word in ls if word not in stopwords]



df = pd.read_excel("C题全部数据/附件3.xlsx")
stopwords = stopwordslist("./stopword.txt")+[' ', '再', '的', '们', '为', '时', '：','市','区','县','小区','请','领导','部门']
df['留言详情'] = df['留言详情'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopwords]))


segs=[]
for i in df['留言详情']:
    line = clearTxt(i)
    seg_line = sent2word(line)
    segs.append(seg_line)
df['切割后留言']=segs

df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ysj\AppData\Local\Temp\jieba.cache
Loading model cost 0.676 seconds.
Prefix dict has been built succesfully.


,留言编号,留言用户,留言主题,留言时间,留言详情,反对数,点赞数,切割后留言
0,188006,A000102948,A3区一米阳光婚纱艺术摄影是否合法纳税了？,2019/2/28 11:25:05,\n \t \t \t \t \t \n \t \t \t \t \t 座落在 A3 区联 ...,0,0,座落在 区联 丰路 米兰 春天 栋 一家 名叫 一米阳光 婚纱 艺术摄影 影楼 年单 工作室...
1,188007,A00074795,咨询A6区道路命名规划初步成果公示和城乡门牌问题,2019/2/14 20:00:00,\n \t \t \t \t \t \n \t \t \t \t \t A6 道路 命名 规...,0,1,道路 命名 规划 初步 成果 公示 文件 转化 正式 成果 希望 加快 路 名 规范 道路 ...
2,188031,A00040066,反映A7县春华镇金鼎村水泥路、自来水到户的问题,2019/7/19 18:19:54,\n \t \t \t \t \t \n \t \t \t \t \t 系 春华 镇金鼎村 ...,0,1,系 春华 镇金鼎村 七里 组 村民 不知 相关 水泥路 到户 政策 自来水 到户 政策 政府...
3,188039,A00081379,A2区黄兴路步行街大古道巷住户卫生间粪便外排,2019/8/19 11:48:23,\n \t \t \t \t \t \n \t \t \t \t \t 靠近 黄兴路 步行街...,0,1,靠近 黄兴路 步行街 城 南路 街道 古道 巷 一步 两 搭桥 停车场 东面 围墙 外 第一...
4,188059,A00028571,A市A3区中海国际社区三期与四期中间空地夜间施工噪音扰民,2019/11/22 16:54:42,\n \t \t \t \t \t \n \t \t \t \t \t A3 中海 国际 社...,0,0,中海 国际 社区 三期 四期 蓝天 璞洲 幼儿园 旁边 块 空地 处于 三不管 状态 物业 ...


In [3]:
documents=df['切割后留言']

words_ls = []
for text in documents:
    words =remove_stopwords([w.word for w in jp.cut(text)])
    words_ls.append(words)
# 生成语料词典
dictionary = corpora.Dictionary(words_ls)
# 生成稀疏向量集
corpus = [dictionary.doc2bow(words) for words in words_ls]
# LDA模型，num_topics设置聚类数，即最终主题的数量
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)
# 展示每个主题的前5的词语
for topic in lda.print_topics(num_words=5):
    print(topic[0],":",topic[1])
# 推断每个语料库中的主题类别
print('推断：')

topic_ids=[]
for e, values in enumerate(lda.inference(corpus)[0]):
    topic_val = 0
    topic_id = 0
    for tid, val in enumerate(values):
        if val > topic_val:
            topic_val = val
            topic_id = tid
    topic_ids.append(topic_id)
    print(topic_id, '->', documents[e])
    
df['预测类别']=topic_ids

0 : 0.009*"居民" + 0.009*"噪音" + 0.007*"栋" + 0.006*"发展" + 0.005*"业主"
1 : 0.010*"交易" + 0.010*"商品交易" + 0.006*"征收" + 0.006*"证照" + 0.006*"年"
2 : 0.009*"业主" + 0.007*"社区" + 0.006*"年" + 0.005*"违规" + 0.005*"相关"
3 : 0.020*"居民" + 0.014*"影响" + 0.013*"晚上" + 0.008*"道" + 0.008*"生活"
4 : 0.015*"业主" + 0.013*"年" + 0.013*"开发商" + 0.007*"政府" + 0.007*"装修"
5 : 0.016*"公司" + 0.013*"出借" + 0.011*"平台" + 0.009*"立案" + 0.009*"资金"
6 : 0.010*"训练" + 0.006*"星沙" + 0.005*"说" + 0.005*"占据" + 0.005*"拐"
7 : 0.014*"年" + 0.007*"号" + 0.006*"说" + 0.005*"系统" + 0.005*"记录"
8 : 0.023*"孩子" + 0.016*"幼儿园" + 0.008*"小学" + 0.007*"学校" + 0.006*"工作"
9 : 0.018*"业主" + 0.008*"开发商" + 0.007*"政府" + 0.007*"施工" + 0.007*"年"
10 : 0.009*"公交" + 0.006*"政策" + 0.004*"城管局" + 0.004*"相关" + 0.004*"情况"
11 : 0.012*"路" + 0.009*"我家" + 0.009*"年" + 0.006*"税务" + 0.005*"说"
12 : 0.009*"套餐" + 0.006*"垃圾" + 0.006*"办理" + 0.006*"盘" + 0.005*"住院"
13 : 0.052*"业主" + 0.026*"物业" + 0.011*"物业管理" + 0.009*"委员会" + 0.008*"物业公司"
14 : 0.012*"年" + 0.009*"政府" + 0.007*"路" + 0.007*"建设" + 0.006*"

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
df

,留言编号,留言用户,留言主题,留言时间,留言详情,反对数,点赞数,切割后留言,预测类别
0,188006,A000102948,A3区一米阳光婚纱艺术摄影是否合法纳税了？,2019/2/28 11:25:05,\n \t \t \t \t \t \n \t \t \t \t \t 座落在 A3 区联 ...,0,0,座落在 区联 丰路 米兰 春天 栋 一家 名叫 一米阳光 婚纱 艺术摄影 影楼 年单 工作室...,15
1,188007,A00074795,咨询A6区道路命名规划初步成果公示和城乡门牌问题,2019/2/14 20:00:00,\n \t \t \t \t \t \n \t \t \t \t \t A6 道路 命名 规...,0,1,道路 命名 规划 初步 成果 公示 文件 转化 正式 成果 希望 加快 路 名 规范 道路 ...,9
2,188031,A00040066,反映A7县春华镇金鼎村水泥路、自来水到户的问题,2019/7/19 18:19:54,\n \t \t \t \t \t \n \t \t \t \t \t 系 春华 镇金鼎村 ...,0,1,系 春华 镇金鼎村 七里 组 村民 不知 相关 水泥路 到户 政策 自来水 到户 政策 政府...,14
3,188039,A00081379,A2区黄兴路步行街大古道巷住户卫生间粪便外排,2019/8/19 11:48:23,\n \t \t \t \t \t \n \t \t \t \t \t 靠近 黄兴路 步行街...,0,1,靠近 黄兴路 步行街 城 南路 街道 古道 巷 一步 两 搭桥 停车场 东面 围墙 外 第一...,9
4,188059,A00028571,A市A3区中海国际社区三期与四期中间空地夜间施工噪音扰民,2019/11/22 16:54:42,\n \t \t \t \t \t \n \t \t \t \t \t A3 中海 国际 社...,0,0,中海 国际 社区 三期 四期 蓝天 璞洲 幼儿园 旁边 块 空地 处于 三不管 状态 物业 ...,9
5,188073,A909164,A3区麓泉社区单方面改变麓谷明珠小区6栋架空层使用性质,2019/3/11 11:40:42,\n \t \t \t \t \t \n \t \t \t \t \t 麓 泉 社区 麓 谷...,0,0,麓 泉 社区 麓 谷 明珠 栋 居民 近期 感觉 震惊 伤心 购房 签合同 办 产权 提及 ...,0
6,188074,A909092,A2区富绿新村房产的性质是什么？,2019/1/31 20:17:32,\n \t \t \t \t \t \n \t \t \t \t \t 〝 二高 一部 发出...,0,0,〝 二高 一部 发出 非法 集资 打击 通知 中是 金融 犯罪 通知 中 点 特征 中 欺骗...,2
7,188119,A00035029,对A市地铁违规用工问题的质疑,2019/5/27 16:04:44,\n \t \t \t \t \t \n \t \t \t \t \t 一名 地铁站 上班 ...,0,0,一名 地铁站 上班 安检员 中介 公司 介绍 上班 安检员 岗位 分 两个 班次 一个班 早...,16
8,188170,A88011323,A市6路公交车随意变道通行,2019/12/23 8:50:24,\n \t \t \t \t \t \n \t \t \t \t \t 12 21 下午 1...,0,0,下午 分 许路 公交车 司机 座位 旁边 汽车 编号 大道 金星 路口 西往东 辅道 左 拐...,18
9,188249,A00084085,A3区保利麓谷林语桐梓坡路与麓松路交汇处地铁凌晨2点施工扰民,2019/9/17 4:25:00,\n \t \t \t \t \t \n \t \t \t \t \t 保利 麓 谷林语 桐...,0,0,保利 麓 谷林语 桐梓 坡路 与麓 松路 交汇处 地铁 凌晨 点 施工 噪音 扰民 家里 老...,3


In [1]:
import jieba
import pandas as pd
import codecs
import string
import re
from sklearn.metrics import silhouette_score,silhouette_samples
import matplotlib.cm as cm
import numpy as np
import jieba.posseg as jp
from gensim import corpora, models

In [2]:
data=pd.read_csv('./泰迪杯第二题关键词.csv')
data.head()

,留言主题,clean_review,cut_review,地区,关键词,留言详情中的实体,留言主题中的实体
0,请求公开A1区东成大厦申报建设成本资料,请求公开A1区东成大厦申报建设成本资料,请求 公开 A1区 东成 大厦 申报 建设 成本 资料,A1区,"['A1区', '东成', '大厦', '申报', '请求']","['西地省经济学院', '钱供', '北京']",['A1区东成大厦']
1,请对A1区华年世嘉酒店及其保安队长立案调查,请对A1区华年世嘉酒店及其保安队长立案调查,请 A1区 华年 世嘉 酒店 保安队 长 立案 调查,A1区,"['华年', 'A1区', '世嘉', '保安队', '立案']",['江苏'],['A1区']
2,投诉A1区公寓幼儿园招生不透明，存在暗箱操作,投诉A1区公寓幼儿园招生不透明存在暗箱操作,投诉 A1区 公寓 幼儿园 招生 透明 暗箱 操作,A1区,"['A1区', '暗箱', '招生', '幼儿园', '公寓']","['A3区', '溪湖', '溪湖', 'A3区', 'A3区', '溪湖', '溪湖', ...",['A1区']
3,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区 沁园 小区 C5 栋 污水 水管 破裂 一个月,A1区,"['A1区', '沁园', 'C5', '水管', '污水']","['朱琦梦', 'A市A6区', 'A市', 'A市']",['A1区沁园']
4,A1区扬帆小区夜市占据了消防通道！,A1区扬帆小区夜市占据了消防通道,A1区 扬帆 小区 夜市 占据 消防通道,A1区,"['A1区', '消防通道', '夜市', '扬帆', '小区']",['排气口'],['A1区']


In [15]:
documents=data['留言详情中的实体']

words_ls = []
for text in documents:
    words =[w.word for w in jp.cut(text)]
    words_ls.append(words)
# 生成语料词典
dictionary = corpora.Dictionary(words_ls)
# 生成稀疏向量集
corpus = [dictionary.doc2bow(words) for words in words_ls]
# LDA模型，num_topics设置聚类数，即最终主题的数量
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50)
# 展示每个主题的前5的词语
#for topic in lda.print_topics(num_words=5):
    #print(topic[0],":",topic[1])
# 推断每个语料库中的主题类别
#print('推断：')

topic_ids=[]
for e, values in enumerate(lda.inference(corpus)[0]):
    topic_val = 0
    topic_id = 0
    for tid, val in enumerate(values):
        if val > topic_val:
            topic_val = val
            topic_id = tid
    topic_ids.append(topic_id)
    #print(topic_id, '->', documents[e])
    
data['预测类别']=topic_ids

In [16]:
data.head()

,留言主题,clean_review,cut_review,地区,关键词,留言详情中的实体,留言主题中的实体,预测类别
0,请求公开A1区东成大厦申报建设成本资料,请求公开A1区东成大厦申报建设成本资料,请求 公开 A1区 东成 大厦 申报 建设 成本 资料,A1区,"['A1区', '东成', '大厦', '申报', '请求']","['西地省经济学院', '钱供', '北京']",['A1区东成大厦'],49
1,请对A1区华年世嘉酒店及其保安队长立案调查,请对A1区华年世嘉酒店及其保安队长立案调查,请 A1区 华年 世嘉 酒店 保安队 长 立案 调查,A1区,"['华年', 'A1区', '世嘉', '保安队', '立案']",['江苏'],['A1区'],1
2,投诉A1区公寓幼儿园招生不透明，存在暗箱操作,投诉A1区公寓幼儿园招生不透明存在暗箱操作,投诉 A1区 公寓 幼儿园 招生 透明 暗箱 操作,A1区,"['A1区', '暗箱', '招生', '幼儿园', '公寓']","['A3区', '溪湖', '溪湖', 'A3区', 'A3区', '溪湖', '溪湖', ...",['A1区'],23
3,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区 沁园 小区 C5 栋 污水 水管 破裂 一个月,A1区,"['A1区', '沁园', 'C5', '水管', '污水']","['朱琦梦', 'A市A6区', 'A市', 'A市']",['A1区沁园'],6
4,A1区扬帆小区夜市占据了消防通道！,A1区扬帆小区夜市占据了消防通道,A1区 扬帆 小区 夜市 占据 消防通道,A1区,"['A1区', '消防通道', '夜市', '扬帆', '小区']",['排气口'],['A1区'],11


In [29]:
a=data.groupby('预测类别',as_index=False)['留言主题'].agg(sum)
a

,预测类别,留言主题
0,0,A5区劳动东路魅力之城小区一楼的夜宵摊严重污染附近的空气，急需处理！A7县旺德府凯悦国际小区...
1,1,请对A1区华年世嘉酒店及其保安队长立案调查A1区东方家园土地被贪，农民维权艰难A1区A9市河...
2,2,A1区定王台派出所边上危房存在安全隐患A1区辉煌国际城居民楼下商铺违法开饭店，老百姓从何谈安...
3,3,A7县泉塘第三小学对面两文具店摆了多台游戏机，难道没人管吗？A市搜门面网是个骗局
4,5,A1区A1区苑附近的报刊亭为何不准经营A1区马王堆沁园小区b区停车位被个人占用了！A1区朝阳...
5,6,A1区沁园小区C5栋污水水管破裂将近一个月了反映A1区五一中路粮贸大厦高空坠物问题A1区解放...
6,8,A1区金楚北苑物业私自撤离，严重影响业主生活A1区辉煌国际城二期居民楼下商铺没有配套专用烟道...
7,9,A2区瑞富苑未办齐相关出售许可证就强制业主收房！A2区丽发新城附近违规乱建混凝土搅拌站谁来监...
8,11,A1区扬帆小区夜市占据了消防通道！A1区辉煌国际城二期商铺被开发商违法出租给别人做餐饮反映A...
9,13,A1区朝阳生鲜市场一店铺占道经营，有安全隐患A1区东岸街道为什么不公开征地补偿明细A2区星语...


In [32]:
length=[]
for i in range(len(a)):
    word_len=len(a['留言主题'][i])
    length.append(word_len)
a['长度']=length
a.head()

,预测类别,留言主题,长度
0,0,A5区劳动东路魅力之城小区一楼的夜宵摊严重污染附近的空气，急需处理！A7县旺德府凯悦国际小区...,127
1,1,请对A1区华年世嘉酒店及其保安队长立案调查A1区东方家园土地被贪，农民维权艰难A1区A9市河...,5069
2,2,A1区定王台派出所边上危房存在安全隐患A1区辉煌国际城居民楼下商铺违法开饭店，老百姓从何谈安...,727
3,3,A7县泉塘第三小学对面两文具店摆了多台游戏机，难道没人管吗？A市搜门面网是个骗局,40
4,5,A1区A1区苑附近的报刊亭为何不准经营A1区马王堆沁园小区b区停车位被个人占用了！A1区朝阳...,5042


In [35]:
a=a.sort_values('长度',ascending=False)
a.head()

,预测类别,留言主题,长度
12,23,投诉A1区公寓幼儿园招生不透明，存在暗箱操作反映A1区兴和社区厕所问题A1区兴和社区火星街道...,16873
5,6,A1区沁园小区C5栋污水水管破裂将近一个月了反映A1区五一中路粮贸大厦高空坠物问题A1区解放...,11409
11,22,A1区东岸锦城A、B区交界处的多家铝合金门窗加工店发出刺耳的切割声A1区五一东村老小区绿化带...,10222
25,49,请求公开A1区东成大厦申报建设成本资料A1区美漾酒店无证违法施工，强行改造永华大厦7楼房屋结...,9979
23,43,恳请A1区政府将“永利金猪”犯罪嫌疑人捉拿归案A1区火星镇兴和社区没有提供供老百姓休闲的地方...,9884


In [7]:
import pandas as pd

data=pd.read_csv('./主谓宾关键词.csv')
data.head()

,留言主题,clean_review,cut_review,地区,主谓宾
0,请求公开A1区东成大厦申报建设成本资料,请求公开A1区东成大厦申报建设成本资料,请求 公开 A1 区东成 大厦 申报 建设 成本 资料,A1区,"大厦请求公开,（副词：）"
1,请对A1区华年世嘉酒店及其保安队长立案调查,请对A1区华年世嘉酒店及其保安队长立案调查,请 A1 区华年 世嘉 酒店 保安队 长 立案 调查,A1区,"酒店请立案,（副词：对）"
2,投诉A1区公寓幼儿园招生不透明，存在暗箱操作,投诉A1区公寓幼儿园招生不透明存在暗箱操作,投诉 A1 区 公寓 幼儿园 招生 透明 暗箱 操作,A1区,"投诉存在操作,（副词：不）"
3,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区沁园小区C5栋污水水管破裂将近一个月了,A1 区沁园 小区 C5 栋 污水 水管 破裂 一个月,A1区,"水管破裂,（副词：）"
4,A1区扬帆小区夜市占据了消防通道！,A1区扬帆小区夜市占据了消防通道,A1 区 扬帆 小区 夜市 占据 消防通道,A1区,"夜市占据消防通道,（副词：）"


In [2]:
import jieba
import codecs
import string
import re
from sklearn.metrics import silhouette_score,silhouette_samples
import matplotlib.cm as cm
import numpy as np
import jieba.posseg as jp
from gensim import corpora, models

In [16]:
# 清洗文本
def clearTxt(line:str):
    if(line != ''):
        line = line.strip()
        # 去除文本中的中文符号和英文符号
        rule = re.sub("[a-zA-Z0-9]","",line)
        line = re.sub("[\s+\.\!\/_,$%^*(+\"\'；：“”．]+|[+——！，。？?、~@#￥%……&*（）]+", "",rule)
        return line
    return None

#文本切割
def sent2word(line):
    segList = jieba.cut(line,cut_all=False)
    segSentence = ''
    for word in segList:
        if word != '\t':
            segSentence += word + " "
    return segSentence.strip()

def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding="GB18030").readlines()]  
    return stopwords  


def remove_stopwords(ls):  # 去除停用词
    return [word for word in ls if word not in stopwords]




stopwords = stopwordslist("./stopword.txt")+[' ', '再', '的', '们', '为', '时', '：','市','区','县','小区','请','领导']
data['切割后主谓宾'] = data['主谓宾'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopwords]))


segs=[]
for i in data['切割后主谓宾']:
    line = clearTxt(i)
    seg_line = sent2word(line)
    seg_line=re.sub('副词','',seg_line)
    segs.append(seg_line)
data['切割后主谓宾']=segs

data.head()

,留言主题,clean_review,cut_review,地区,主谓宾,切割后主谓宾
0,请求公开A1区东成大厦申报建设成本资料,请求公开A1区东成大厦申报建设成本资料,请求 公开 A1 区东成 大厦 申报 建设 成本 资料,A1区,"大厦请求公开,（副词：）",大厦 请求 公开
1,请对A1区华年世嘉酒店及其保安队长立案调查,请对A1区华年世嘉酒店及其保安队长立案调查,请 A1 区华年 世嘉 酒店 保安队 长 立案 调查,A1区,"酒店请立案,（副词：对）",酒店 立案
2,投诉A1区公寓幼儿园招生不透明，存在暗箱操作,投诉A1区公寓幼儿园招生不透明存在暗箱操作,投诉 A1 区 公寓 幼儿园 招生 透明 暗箱 操作,A1区,"投诉存在操作,（副词：不）",投诉 操作
3,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区沁园小区C5栋污水水管破裂将近一个月了,A1 区沁园 小区 C5 栋 污水 水管 破裂 一个月,A1区,"水管破裂,（副词：）",水管 破裂
4,A1区扬帆小区夜市占据了消防通道！,A1区扬帆小区夜市占据了消防通道,A1 区 扬帆 小区 夜市 占据 消防通道,A1区,"夜市占据消防通道,（副词：）",夜市 占据 消防通道


In [21]:
documents=data['cut_review'].astype(str)

words_ls = []
for text in documents:
    words =remove_stopwords([w.word for w in jp.cut(text)])
    words_ls.append(words)
# 生成语料词典
dictionary = corpora.Dictionary(words_ls)
# 生成稀疏向量集
corpus = [dictionary.doc2bow(words) for words in words_ls]
# LDA模型，num_topics设置聚类数，即最终主题的数量
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50)
# 展示每个主题的前5的词语
# for topic in lda.print_topics(num_words=5):
#     print(topic[0],":",topic[1])
# # 推断每个语料库中的主题类别
# print('推断：')

topic_ids=[]
for e, values in enumerate(lda.inference(corpus)[0]):
    topic_val = 0
    topic_id = 0
    for tid, val in enumerate(values):
        if val > topic_val:
            topic_val = val
            topic_id = tid
    topic_ids.append(topic_id)
    #print(topic_id, '->', documents[e])
    
data['预测类别']=topic_ids

In [29]:
data.head()

,留言主题,clean_review,cut_review,地区,主谓宾,切割后主谓宾,预测类别
0,请求公开A1区东成大厦申报建设成本资料,请求公开A1区东成大厦申报建设成本资料,请求 公开 A1 区东成 大厦 申报 建设 成本 资料,A1区,"大厦请求公开,（副词：）",大厦 请求 公开,11
1,请对A1区华年世嘉酒店及其保安队长立案调查,请对A1区华年世嘉酒店及其保安队长立案调查,请 A1 区华年 世嘉 酒店 保安队 长 立案 调查,A1区,"酒店请立案,（副词：对）",酒店 立案,33
2,投诉A1区公寓幼儿园招生不透明，存在暗箱操作,投诉A1区公寓幼儿园招生不透明存在暗箱操作,投诉 A1 区 公寓 幼儿园 招生 透明 暗箱 操作,A1区,"投诉存在操作,（副词：不）",投诉 操作,22
3,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区沁园小区C5栋污水水管破裂将近一个月了,A1 区沁园 小区 C5 栋 污水 水管 破裂 一个月,A1区,"水管破裂,（副词：）",水管 破裂,39
4,A1区扬帆小区夜市占据了消防通道！,A1区扬帆小区夜市占据了消防通道,A1 区 扬帆 小区 夜市 占据 消防通道,A1区,"夜市占据消防通道,（副词：）",夜市 占据 消防通道,1


In [31]:
data['长度']=[1 for i in range(len(data))]
data.head()

,留言主题,clean_review,cut_review,地区,主谓宾,切割后主谓宾,预测类别,长度
0,请求公开A1区东成大厦申报建设成本资料,请求公开A1区东成大厦申报建设成本资料,请求 公开 A1 区东成 大厦 申报 建设 成本 资料,A1区,"大厦请求公开,（副词：）",大厦 请求 公开,11,1
1,请对A1区华年世嘉酒店及其保安队长立案调查,请对A1区华年世嘉酒店及其保安队长立案调查,请 A1 区华年 世嘉 酒店 保安队 长 立案 调查,A1区,"酒店请立案,（副词：对）",酒店 立案,33,1
2,投诉A1区公寓幼儿园招生不透明，存在暗箱操作,投诉A1区公寓幼儿园招生不透明存在暗箱操作,投诉 A1 区 公寓 幼儿园 招生 透明 暗箱 操作,A1区,"投诉存在操作,（副词：不）",投诉 操作,22,1
3,A1区沁园小区C5栋污水水管破裂将近一个月了,A1区沁园小区C5栋污水水管破裂将近一个月了,A1 区沁园 小区 C5 栋 污水 水管 破裂 一个月,A1区,"水管破裂,（副词：）",水管 破裂,39,1
4,A1区扬帆小区夜市占据了消防通道！,A1区扬帆小区夜市占据了消防通道,A1 区 扬帆 小区 夜市 占据 消防通道,A1区,"夜市占据消防通道,（副词：）",夜市 占据 消防通道,1,1


In [36]:
a=data.groupby('预测类别',as_index=False)['长度'].sum()
a=a.sort_values('长度',ascending=False)
a.head()

,预测类别,长度
12,12,145
16,16,135
7,7,135
22,22,119
9,9,114


In [40]:
b=data[data['预测类别']==22]
b

,留言主题,clean_review,cut_review,地区,主谓宾,切割后主谓宾,预测类别,长度
2,投诉A1区公寓幼儿园招生不透明，存在暗箱操作,投诉A1区公寓幼儿园招生不透明存在暗箱操作,投诉 A1 区 公寓 幼儿园 招生 透明 暗箱 操作,A1区,"投诉存在操作,（副词：不）",投诉 操作,22,1
44,A1区大同瑞致小学违规使用家校通平台,A1区大同瑞致小学违规使用家校通平台,A1 区 大同 瑞致 小学 违规 家校通 平台,A1区,"小学违规平台,（副词：）",小学 违规 平台,22,1
53,A1区同鑫家园小区厕所、地漏反水,A1区同鑫家园小区厕所地漏反水,A1 区同鑫 家园 小区 厕所 地漏 反水,A1区,"地漏反水,（副词：）",地漏 反水,22,1
71,A1区新安安置小区6栋三单元的下水井堵塞了,A1区新安安置小区6栋三单元的下水井堵塞了,A1 区 新安 安置 小区 栋 单元 水井 堵塞,A1区,"下水井,（副词：）",水井,22,1
104,A1区紫来阁小区公共用地被侵占,A1区紫来阁小区公共用地被侵占,A1 区紫来 阁 小区 公共 用地 侵占,A1区,"侵占用地,（副词：被）",侵占 用地,22,1
129,A1区地铁五号线马王堆站经常超时作业，影响居民正常休息,A1区地铁五号线马王堆站经常超时作业影响居民正常休息,A1 区 地铁 五号线 马王堆 站 超时 作业 影响 居民 休息,A1区,"站超时休息,（副词：经常）",站 超时 休息,22,1
348,A2区明德启南中学克扣教师绩效奖金,A2区明德启南中学克扣教师绩效奖金,A2 区 明德 启南 中学 克扣 教师 绩效奖金,A2区,"中学克扣绩效奖金,（副词：）",中学 克扣 绩效奖金,22,1
404,A3区麓谷一师范安置小区抽烟机影响居民生活睡眠,A3区麓谷一师范安置小区抽烟机影响居民生活睡眠,A3 区麓 谷 师范 安置 小区 抽烟 机 影响 居民 生活 睡眠,A3区,"谷安置小区,（副词：）",谷 安置,22,1
425,A3区佳睿新苗幼儿园涉嫌乱收费,A3区佳睿新苗幼儿园涉嫌乱收费,A3 区佳 睿 新苗 幼儿园 涉嫌 乱收费,A3区,"幼儿园涉嫌乱收费,（副词：）",幼儿园 涉嫌 乱收费,22,1
498,A3区谷山庭院14栋社区艾瑞弗艺术培训学校涉黑,A3区谷山庭院14栋社区艾瑞弗艺术培训学校涉黑,A3 区谷山 庭院 14 栋 社区 艾瑞弗 艺术 培训 学校 涉黑,A3区,"学校涉黑艺术,（副词：）",学校 涉黑 艺术,22,1


In [34]:
table=pd.pivot_table(data,index=["预测类别","留言主题"],columns=['长度'],aggfunc=sum)
table

clean_review  \
长度                                                                         1   
预测类别 留言主题                                                                      
0    A1区鑫科明珠小区学生读不了育才二小                                   A1区鑫科明珠小区学生读不了育才二小   
     A3区万翔国际大酒店西侧装外墙电梯，有非常大的安全隐患                  A3区万翔国际大酒店西侧装外墙电梯有非常大的安全隐患   
     A3区含浦镇九丰村有人破坏水土环境                                     A3区含浦镇九丰村有人破坏水土环境   
     A3区天麓尚层二期的消防通道被关了                                     A3区天麓尚层二期的消防通道被关了   
     A3区天麓尚层小区底商占用人行走廊违建                                 A3区天麓尚层小区底商占用人行走廊违建   
     A3区金星路每天车流量非常大，门口斑马线交通事故频发                    A3区金星路每天车流量非常大门口斑马线交通事故频发   
     A4区A4区寺围墙下摆放的报废共享单车影响市容                         A4区A4区寺围墙下摆放的报废共享单车影响市容   
     A4区九尾冲没有斑马线，行人直接在车流中插空过马路                      A4区九尾冲没有斑马线行人直接在车流中插空过马路   
     A4区凯乐国际城周边车辆乱停乱放问题依然没解决                         A4区凯乐国际城周边车辆乱停乱放问题依然没解决   
     A4区凯乐国际城太平路路灯失修大半年未解决，该向谁问责                  A4区凯乐国际城太平路路灯失修大半年未解决该向谁问责   
     A4区培元桥科技情报站小区成了外卖集中点                               A4区培元桥科技情报站小区成了外卖集中点   
     A4区大塘村陈家冲组组长强行私填鱼塘田地！                              A4区大塘村陈家冲组组长强行私填鱼塘田地   
     A4区太阳山森林公园山顶上的塔年久失修，有危险                          A4区太阳山森林公园山顶上的塔年久失修有危险   
     A4区沙坪楚绣老街小摊小贩霸地经营，车辆乱停乱放                        A4区沙坪楚绣老街小摊小贩霸地经营车辆乱停乱放   
     A6区乐儿湾婴童健康理疗水育中心暂停营业还不退费                       A6区乐儿湾婴童健康理疗水育中心暂停营业还不退费   
     A6区月亮岛路11万伏高压线没用地埋方式铺设                           A6区月亮岛路11万伏高压线没用地埋方式铺设   
     A6区润和长郡小区架空层的养老中心该不该建？                            A6区润和长郡小区架空层的养老中心该不该建   
     A7县北山镇新中村什么时候通公交车呢？                                  A7县北山镇新中村什么时候通公交车呢   
     A7县开元路深业睿城施工噪音太大                                       A7县开元路深业睿城施工噪音太大   
     A7县星沙医院如此明目张胆偷税漏税？                                    A7县星沙医院如此明目张胆偷税漏税   
     A7县楚龙西路保利香槟国际周边的交通问题如何解决                       A7县楚龙西路保利香槟国际周边的交通问题如何解决   
     A7县泉塘泉星物流园a区7栋锦和超市后街依然摆放游戏赌博机             A7县泉塘泉星物流园a区7栋锦和超市后街依然摆放游戏赌博机   
     A7县泉塘泉星物流园a区7栋锦和超市后街公然摆放游戏赌博机             A7县泉塘泉星物流园a区7栋锦和超市后街公然摆放游戏赌博机   
     A7县泉塘泉星物流园废弃油烟排放管道存在安全隐患                       A7县泉塘泉星物流园废弃油烟排放管道存在安全隐患   
     A7县特立路与东四路口晚高峰太堵，建议调整信号灯配时                    A7县特立路与东四路口晚高峰太堵建议调整信号灯配时   
     A7县聚友轮胎厂污染环境                                               A7县聚友轮胎厂污染环境   
     A7县茂中城已售商铺被查封，业主心急如焚                                A7县茂中城已售商铺被查封业主心急如焚   
     A8县市大屯营乡居民家房屋后架设了一条220万伏的高压线！              A8县市大屯营乡居民家房屋后架设了一条220万伏的高压线   
     A8县科目三补考费要600一次，合理吗？                                 A8县科目三补考费要600一次合理吗   
     A市26路始发站黎郡社区北停车杂乱无章                                 A市26路始发站黎郡社区北停车杂乱无章   
...                                                                      ...   
49   咨询A市潇楚卡异地优惠问题                                             咨询A市潇楚卡异地优惠问题   
     在A市摇号买房，摇到了不买的话会有什么影响吗？                           在A市摇号买房摇到了不买的话会有什么影响吗   
     外地二手车想要迁入A市，对于排放是否有限制？                             外地二手车想要迁入A市对于排放是否有限制   
     对A3区银双路、楚岳路交汇路口交通环境的意见                            对A3区银双路楚岳路交汇路口交通环境的意见   
     建议A7县在漓楚路和东六路交汇处建地下通道或人行天桥                   建议A7县在漓楚路和东六路交汇处建地下通道或人行天桥   
     建议A7县杨慧开纪念馆尽快建设官方网站和微信公众号                     建议A7县杨慧开纪念馆尽快建设官方网站和微信公众号   
     建议A市A3区书院取消门票收费                                         建议A市A3区书院取消门票收费   
     建议A市修复西地省和平解放条文签定地，打造红色旅游名片                  建议A市修复西地省和平解放条文签定地打造红色旅游名片   
     建议A市经开区收回东六路恒天九五工厂地块，打造商业综合体                建议A市经开区收回东六路恒天九五工厂地块打造商业综合体   
     建议取消A7县盼盼路（东六路至东四路段）的三条人行道                     建议取消A7县盼盼路东六路至东四路段的三条人行道   
     建议合理配置A市望仙路交通信号灯及标识标线                             建议合理配置A市望仙路交通信号灯及标识标线   
     建议在A市咸嘉湖西路馨香雅苑路口装摄像头                               建议在A市咸嘉湖西路馨香雅苑路口装摄像头   
     微博博主“考拉·Koala”和微信公众号“爱玩客ivankr”涉嫌网络诈骗  微博博主考拉Koala和微信公众号爱玩客ivankr涉嫌网络诈骗   
     想请西地省计算机软件资格考试办公室出具核查证明怎么这么难？              想请西地省计算机软件资格考试办公室出具核查证明怎么这么难   
     投诉A市银行白马挢、流沙河两大支行侵权                                  投诉A市银行白马挢流沙河两大支行侵权   
     投诉西地省A2区实业不准未购买车位的业主车辆进入地下停车场             投诉西地省A2区实业不准未购买车位的业主车辆进入地下停车场   
     新房在A6区雷峰政务超市交物业维修金为何只能用A市银行的卡？            新房在A6区雷峰政务超市交物业维修